In [38]:
import requests
import keys
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [39]:
# def fetch(fields, max_per_call=500):
#     base_url = 'https://api.igdb.com/v4/age_ratings'
#     headers = {
#         'Client-ID': keys.API_CLIENT_ID,  
#         'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
#     }
    
#     all_data = []
#     offset = 0
    
#     while True:
#         data = f"fields {','.join(fields)}; limit {max_per_call}; offset {offset};"
#         response = requests.post(base_url, headers=headers, data=data)
        
#         if response.status_code != 200:
#             print(f"Error: {response.status_code}, {response.text}")
#             break
        
#         games = response.json()  
        
#         if not games:
#             break
#         all_data.extend(games)  
#         print(f"Scraped {offset+500} games")
#         if len(games) < max_per_call:
#             break
        
#         offset += max_per_call  
    
#     return all_data  


In [40]:
import requests

def fetch():
    base_url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': keys.API_CLIENT_ID,  
        'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
    }
    all_data = []
    offset = 0
    while True:
        # Define the query for the game
        data = f"""fields 
            id,slug, total_rating,
            age_ratings.rating, age_ratings.category,first_release_date,
            game_modes.slug, genres.slug,
            involved_companies.company.slug, involved_companies.developer, involved_companies.publisher,
            keywords.slug, platforms.slug, player_perspectives.slug;
            where rating > 60 & first_release_date>1111977600;
            limit 500;
            offset {offset};
        """
        
        try:
            response = requests.post(base_url, headers=headers, data=data)
            
            if response.status_code != 200:
                print(f"Error: {response.status_code}, {response.text}")
                return
            
            games = response.json()  
            
            if not games:
                print("No game data found.")
                break

            all_data.extend(games)  
            print(f"Scraped {offset+500} games")
            if len(games) < 500:
                break
        
            offset += 500  
        
        except requests.RequestException as e:
            print(f"Request failed: {e}")
            break
    return all_data


In [41]:
fetched=fetch()

Scraped 500 games
Scraped 1000 games
Scraped 1500 games
Scraped 2000 games
Scraped 2500 games
Scraped 3000 games
Scraped 3500 games
Scraped 4000 games
Scraped 4500 games
Scraped 5000 games
Scraped 5500 games
Scraped 6000 games
Scraped 6500 games
Scraped 7000 games
Scraped 7500 games
Scraped 8000 games
Scraped 8500 games
Scraped 9000 games
Scraped 9500 games
Scraped 10000 games
Scraped 10500 games
Scraped 11000 games
Scraped 11500 games
Scraped 12000 games
Scraped 12500 games
Scraped 13000 games
Scraped 13500 games
Scraped 14000 games
Scraped 14500 games
Scraped 15000 games
Scraped 15500 games
Scraped 16000 games
Scraped 16500 games


In [42]:
df=pd.DataFrame(fetched)

In [43]:
df.head()

,id,age_ratings,first_release_date,game_modes,genres,involved_companies,platforms,slug,total_rating,keywords,player_perspectives
0,83466,"[{'id': 38772, 'category': 1, 'rating': 8}, {'...",1579132800,"[{'id': 1, 'slug': 'single-player'}]","[{'id': 9, 'slug': 'puzzle'}, {'id': 15, 'slug...","[{'id': 133617, 'company': {'id': 17587, 'slug...","[{'id': 130, 'slug': 'switch'}]",spider-solitaire,70.000000,NaN,NaN
1,9284,NaN,1417132800,"[{'id': 1, 'slug': 'single-player'}]","[{'id': 8, 'slug': 'platform'}, {'id': 31, 'sl...","[{'id': 117068, 'company': {'id': 5072, 'slug'...","[{'id': 6, 'slug': 'win'}]",wings-of-vi,70.151379,"[{'id': 905, 'slug': 'difficult'}, {'id': 962,...","[{'id': 4, 'slug': 'side-view'}]"
2,46722,NaN,1417392000,"[{'id': 1, 'slug': 'single-player'}]","[{'id': 12, 'slug': 'role-playing-rpg'}, {'id'...","[{'id': 102182, 'company': {'id': 26446, 'slug...","[{'id': 6, 'slug': 'win'}]",touhou-puppet-dance-performance,89.858497,"[{'id': 5, 'slug': 'zombies'}, {'id': 122, 'sl...","[{'id': 3, 'slug': 'bird-view-slash-isometric'}]"
3,129381,NaN,1580169600,"[{'id': 1, 'slug': 'single-player'}]","[{'id': 31, 'slug': 'adventure'}, {'id': 32, '...",NaN,"[{'id': 6, 'slug': 'win'}]",fishy2,70.000000,NaN,NaN
4,104060,"[{'id': 40455, 'category': 1, 'rating': 8}, {'...",1529280000,"[{'id': 1, 'slug': 'single-player'}]","[{'id': 9, 'slug': 'puzzle'}, {'id': 15, 'slug...","[{'id': 133043, 'company': {'id': 9301, 'slug'...","[{'id': 6, 'slug': 'win'}, {'id': 49, 'slug': ...",mushroom-quest,70.000000,"[{'id': 17177, 'slug': 'sokoban'}]","[{'id': 2, 'slug': 'third-person'}, {'id': 3, ..."


In [44]:
category_map = {
    1: 'ESRB', 
    2: 'PEGI', 
    3: 'CERO', 
    4: 'USK', 
    5: 'GRAC', 
    6: 'CLASS_IND', 
    7: 'ACB'
}

rating_map = {
    1: 'Three', 2: 'Seven', 3: 'Twelve', 4: 'Sixteen', 5: 'Eighteen',
    6: 'RP', 7: 'EC', 8: 'E', 9: 'E10', 10: 'T', 11: 'M', 12: 'AO',
    13: 'CERO_A', 14: 'CERO_B', 15: 'CERO_C', 16: 'CERO_D', 17: 'CERO_Z',
    18: 'USK_0', 19: 'USK_6', 20: 'USK_12', 21: 'USK_16', 22: 'USK_18',
    23: 'GRAC_ALL', 24: 'GRAC_Twelve', 25: 'GRAC_Fifteen', 26: 'GRAC_Eighteen',
    27: 'GRAC_TESTING', 28: 'CLASS_IND_L', 29: 'CLASS_IND_Ten', 30: 'CLASS_IND_Twelve',
    31: 'CLASS_IND_Fourteen', 32: 'CLASS_IND_Sixteen', 33: 'CLASS_IND_Eighteen',
    34: 'ACB_G', 35: 'ACB_PG', 36: 'ACB_M', 37: 'ACB_MA15', 38: 'ACB_R18', 39: 'ACB_RC'
}

In [45]:
def transform_age_ratings(age_ratings):
    if pd.isna(age_ratings).all() if isinstance(age_ratings, list) else pd.isna(age_ratings):
        return []
    
    return [
        f"{category_map.get(rating.get('category'), 'Unknown')} - {rating_map.get(rating.get('rating'), 'Unknown')}"
        for rating in age_ratings
        if rating.get('category') in category_map  
    ]

def get_slug(items):
    if pd.isna(items).all() if isinstance(items, list) else pd.isna(items):
        return []
    
    return [
        item.get('slug', 'unknown')  
        for item in items
    ]

def transform_companies(involved_companies):
    if pd.isna(involved_companies).all() if isinstance(involved_companies, list) else pd.isna(involved_companies):
        return []
    
    return [
        involved_company.get('company', {}).get('slug', 'unknown')  
        for involved_company in involved_companies
    ]

def get_time(unix):
    if pd.isna(unix):
        return None  
    return datetime.fromtimestamp(unix)


In [46]:
df['age_ratings'] = df['age_ratings'].apply(transform_age_ratings)

In [47]:
df['game_modes']=df['game_modes'].apply(get_slug)

In [48]:
df['genres']=df['genres'].apply(get_slug)

In [49]:
df['involved_companies']=df['involved_companies'].apply(transform_companies)

In [50]:
df['keywords']=df['keywords'].apply(get_slug)

In [51]:
df['platforms']=df['platforms'].apply(get_slug)

In [52]:
df['player_perspectives']=df['player_perspectives'].apply(get_slug)

In [53]:
df['first_release_date']=df['first_release_date'].apply(get_time)

In [54]:
df.head()

,id,age_ratings,first_release_date,game_modes,genres,involved_companies,platforms,slug,total_rating,keywords,player_perspectives
0,83466,"[ESRB - E, PEGI - Three, GRAC - GRAC_ALL]",2020-01-16 05:30:00,[single-player],"[puzzle, strategy, turn-based-strategy-tbs, ad...",[baltoro-games],[switch],spider-solitaire,70.000000,[],[]
1,9284,[],2014-11-28 05:30:00,[single-player],"[platform, adventure, indie]",[grynsoft],[win],wings-of-vi,70.151379,"[difficult, female-protagonist, action-adventu...",[side-view]
2,46722,[],2014-12-01 05:30:00,[single-player],"[role-playing-rpg, adventure, indie]",[focaslens],[win],touhou-puppet-dance-performance,89.858497,"[zombies, hell, vampire, magic, 2d, pokemon, j...",[bird-view-slash-isometric]
3,129381,[],2020-01-28 05:30:00,[single-player],"[adventure, indie]",[],[win],fishy2,70.000000,[],[]
4,104060,"[ESRB - E, PEGI - Three, GRAC - GRAC_ALL, CLAS...",2018-06-18 05:30:00,[single-player],"[puzzle, strategy, indie, arcade, card-and-boa...","[xitilon, flying-islands-team, drageus-games]","[win, xboxone, switch]",mushroom-quest,70.000000,[sokoban],"[third-person, bird-view-slash-isometric]"


In [55]:
cols=['id','age_ratings','first_release_date','game_modes','genres','involved_companies','keywords','platforms','player_perspectives','total_rating']
data=df[cols]
info=df.drop(columns=cols)

In [56]:
data.head(5)

,id,age_ratings,first_release_date,game_modes,genres,involved_companies,keywords,platforms,player_perspectives,total_rating
0,83466,"[ESRB - E, PEGI - Three, GRAC - GRAC_ALL]",2020-01-16 05:30:00,[single-player],"[puzzle, strategy, turn-based-strategy-tbs, ad...",[baltoro-games],[],[switch],[],70.000000
1,9284,[],2014-11-28 05:30:00,[single-player],"[platform, adventure, indie]",[grynsoft],"[difficult, female-protagonist, action-adventu...",[win],[side-view],70.151379
2,46722,[],2014-12-01 05:30:00,[single-player],"[role-playing-rpg, adventure, indie]",[focaslens],"[zombies, hell, vampire, magic, 2d, pokemon, j...",[win],[bird-view-slash-isometric],89.858497
3,129381,[],2020-01-28 05:30:00,[single-player],"[adventure, indie]",[],[],[win],[],70.000000
4,104060,"[ESRB - E, PEGI - Three, GRAC - GRAC_ALL, CLAS...",2018-06-18 05:30:00,[single-player],"[puzzle, strategy, indie, arcade, card-and-boa...","[xitilon, flying-islands-team, drageus-games]",[sokoban],"[win, xboxone, switch]","[third-person, bird-view-slash-isometric]",70.000000


In [57]:
data.to_csv('data.csv',index=False)

In [ ]:
import requests

def fetch():
    base_url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': keys.API_CLIENT_ID,  
        'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
    }
    all_data = []
    offset = 0
    while True:
        # Define the query for the game
        data = f"""fields 
            id,slug, total_rating,
            age_ratings.rating, age_ratings.category,first_release_date,
            game_modes.slug, genres.slug,
            involved_companies.company.slug, involved_companies.developer, involved_companies.publisher,
            keywords.slug, platforms.slug, player_perspectives.slug;
            where rating > 60 & first_release_date>1111977600;
            limit 500;
            offset {offset};
        """
        
        try:
            response = requests.post(base_url, headers=headers, data=data)
            
            if response.status_code != 200:
                print(f"Error: {response.status_code}, {response.text}")
                return
            
            games = response.json()  
            
            if not games:
                print("No game data found.")
                break

            all_data.extend(games)  
            print(f"Scraped {offset+500} games")
            if len(games) < 500:
                break
        
            offset += 500  
        
        except requests.RequestException as e:
            print(f"Request failed: {e}")
            break
    return all_data


,slug
0,spider-solitaire
1,wings-of-vi
2,touhou-puppet-dance-performance
3,fishy2
4,mushroom-quest
